In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/open-problems-multimodal/sample_submission.csv
/kaggle/input/open-problems-multimodal/train_cite_targets.h5
/kaggle/input/open-problems-multimodal/metadata_cite_day_2_donor_27678.csv
/kaggle/input/open-problems-multimodal/test_multi_inputs.h5
/kaggle/input/open-problems-multimodal/evaluation_ids.csv
/kaggle/input/open-problems-multimodal/train_cite_inputs.h5
/kaggle/input/open-problems-multimodal/train_multi_targets.h5
/kaggle/input/open-problems-multimodal/train_multi_inputs.h5
/kaggle/input/open-problems-multimodal/metadata.csv
/kaggle/input/open-problems-multimodal/test_cite_inputs_day_2_donor_27678.h5
/kaggle/input/open-problems-multimodal/test_cite_inputs.h5
/kaggle/input/open-problems-msci-multiome-sparse-matrices/train_multi_targets_sparse.npz
/kaggle/input/open-problems-msci-multiome-sparse-matrices/test_multi_inputs_sparse.npz
/kaggle/input/open-problems-msci-multiome-sparse-matrices/train_multiome_input_sparse.npz
/kaggle/input/data-process/multi_test_col_name.n

In [2]:
# 导入库
import pandas as pd
import numpy as np
# from tqdm import tqdm,tqdm_notebook 
from tqdm.notebook import tqdm,trange
import gc
import os
import sys
from scipy import sparse
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn.functional as F
from torch import Tensor
!pip install tables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 2.9 MB/s eta 0:00:00


In [3]:
# from sqrt4kaido
class NegativeCorrLoss(torch.nn.Module):
    """Negative correlation loss function for Keras

    Precondition:
    y_true.mean(axis=1) == 0
    y_true.std(axis=1) == 1

    Returns:
    -1 = perfect positive correlation
    1 = totally negative correlation
    """

    def __init__(self):
        super().__init__()

    def forward(self, preds, targets):

        my = torch.mean(preds, dim=1)
        my = torch.tile(torch.unsqueeze(my, dim=1), (1, targets.shape[1]))
        ym = preds - my
        r_num = torch.sum(torch.multiply(targets, ym), dim=1)
        r_den = torch.sqrt(
            torch.sum(torch.square(ym), dim=1) * float(targets.shape[-1])
        )
        r = torch.mean(r_num / r_den)
        return -r
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

In [4]:
class DatasetSparse_cite(torch.utils.data.Dataset):
    def __init__(self,data_x_path,data_y_path):
        """
        """
        self.x = sparse.load_npz(data_x_path) 
        Y = pd.read_hdf(data_y_path).values
        Y -= Y.mean(axis=1).reshape(-1, 1)
        Y /= Y.std(axis=1).reshape(-1, 1)

        self.y = Y
        self.number = self.x.shape[0]
        print('Finish loading data into memory')
        
    def __len__(self):
        return self.number

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    
def collate_fn_sparse_cite(batch):
    """
    """
    index,feats,values,labels = [],[],[],[]
    for idx, (feature, label) in enumerate(batch):
        index+=[idx]*len(feature.indices)
        feats+=feature.indices.tolist()
        values+=feature.data.tolist()
        labels+=[label.tolist()]
    return torch.tensor(index), torch.tensor(feats), torch.tensor(values),torch.tensor(labels)

    

class DatasetSparse_multi(torch.utils.data.Dataset):
    def __init__(self,data_x_path,data_y_path):
        """
        """
        self.x = sparse.load_npz(data_x_path) 
        self.y = sparse.load_npz(data_y_path) 
        self.y_mean = np.load('../input/data-process/multi_y_mean.npy')
        self.y_std = np.load('../input/data-process/multi_y_std.npy')
        self.number = self.x.shape[0]
        
        print('Finish loading data into memory')
        
    def __len__(self):
        return self.number

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx],self.y_mean[idx],self.y_std[idx] 

def collate_fn_sparse_multi(batch):
    """
    """
    index,feats,values,labels = [],[],[],[]
    for idx, (feature, label, label_mean, label_std) in enumerate(batch):
        index+=[idx]*len(feature.indices)
        feats+=feature.indices.tolist()
        values+=feature.data.tolist()
        labels+=((label.toarray()-label_mean)/label_std).tolist()
    return torch.tensor(index), torch.tensor(feats), torch.tensor(values),torch.tensor(labels)


class DatasetSparse_pred(torch.utils.data.Dataset):
    def __init__(self,data_x_path):
        """
        """
        self.pred_x = sparse.load_npz(data_x_path) 
        self.number = self.pred_x.shape[0]
        print('Finish loading data into memory')
    def __len__(self):
        return self.number

    def __getitem__(self,idx):
        return self.pred_x[idx]
    
def collate_fn_sparse_pred(batch):
    """
    """
    index,feats,values,labels = [],[],[],[]
    for idx, feature in enumerate(batch):
        index+=[idx]*len(feature.indices)
        feats+=feature.indices.tolist()
        values+=feature.data.tolist()
    return torch.tensor(index), torch.tensor(feats), torch.tensor(values)

In [5]:
# model define
class LinearReg(torch.nn.Module):
    
    def __init__(self, model_config):
        """
        model_config : {'input_dim':20000,'device':'cpu','output':140,'layer_one':160,'fc_dims':[64,32]}
        """
        super(LinearReg, self).__init__()

        self.input_dim = model_config['input_dim']
        self.device = model_config['device']
        self.output = model_config['output']
        self.layer_one = model_config['layer_one']
        self.fc_dims = model_config['fc_dims']
        self.layer_one_lst = torch.nn.ParameterList()
        
        for index,i in enumerate(range(self.layer_one)):
            self.layer_one_lst.append(torch.nn.Parameter(torch.zeros(1, 1,device=self.device)))
            self.layer_one_lst.append(torch.nn.Parameter(torch.randn(self.input_dim, 1,device=self.device))) # 多1维，从 1 开始
            torch.nn.init.xavier_uniform_(self.layer_one_lst[index * 2+1],gain=1)
        
        num_dim = self.layer_one
        dim = self.layer_one
        
        self.mats = torch.nn.ParameterList()
        for (index, fc_dim) in enumerate(self.fc_dims): # [10,5,1]
            num_dim+=fc_dim
            self.mats.append(torch.nn.Parameter(torch.randn(dim, fc_dim,device=self.device)))
            self.mats.append(torch.nn.Parameter(torch.zeros(1, fc_dim,device=self.device))) # 可以多个 b
            torch.nn.init.kaiming_uniform_(self.mats[index * 2], mode='fan_in', nonlinearity='relu')
            dim = fc_dim
            
        self.output_w = torch.nn.Parameter(torch.randn(num_dim, self.output,device=self.device))
        self.output_b = torch.nn.Parameter(torch.zeros(1, self.output,device=self.device)) # 可以多个 b
        torch.nn.init.kaiming_uniform_(self.output_w, mode='fan_in', nonlinearity='relu')
        
        
            
            
    def first_order(self, batch_size, index, values, bias, weights):
        # type: (int, Tensor, Tensor, Tensor, Tensor) -> Tensor
        size = batch_size # 64
        srcs = weights.view(1, -1).mul(values.view(1, -1)).view(-1) # weight [1234,1] values[1234]  -> 1x1232 -> 1234
        output = torch.zeros(size, dtype=torch.float32,device = self.device) # 64 
        # output=output.to(device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
        output.scatter_add_(0, index, srcs) # dim,index,src  index 是1维   output[index[i]]+=srcs[i] 遍历 index 的 i index:[0,0,0,0,0,\cdots,1,1,\cdots,63,63] e.eg [1234] 
        first = output + bias # [64] # wx+b 一阶
        return first
    
    
    def higher_order(self, batch_size, index, mats):
        for i in range(int(len(mats) / 2)):
            output = torch.relu(output.matmul(mats[i * 2]) + mats[i * 2 + 1]) # 全连接层
    

    def forward(self, batch_size, index, feats, values):
        # type: (int, Tensor, Tensor, Tensor, Tensor) -> Tensor
        # batch_size : 64
        ret_lst = []
        for idx,i in enumerate(range(self.layer_one)):
            batch_first = F.embedding(feats, self.layer_one_lst[idx * 2+1]) # weights [100000,1]  feats: [a_1,a_1,\cdots,a_64,a_64] e.g [1234] batch_first: [1234,1]
            ret_lst.append(self.first_order(batch_size, index, values, self.layer_one_lst[idx * 2],batch_first).view(-1,1)) # [64,1]
        output = torch.relu(torch.concat(ret_lst,axis=1))    
        
        ret_lst_two = [output]
        for i in range(int(len(self.mats) / 2)):
            output = torch.relu(output.matmul(self.mats[i * 2]) + self.mats[i * 2 + 1])
            ret_lst_two.append(output) # 全连接层
        
        second = torch.relu(torch.concat(ret_lst_two,axis=1))   
            
        return second.matmul(self.output_w)+self.output_b

    @torch.jit.export
    def get_name(self):
        return "LinearReg"

In [6]:
train  = 'multi'

if train == 'cite':
    # train cite
    train_set = DatasetSparse_cite('../input/torch-demo/train_cite_inputs_sparse.npz','../input/open-problems-multimodal/train_cite_targets.h5')
    train_set_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, 
                                                   num_workers=2,drop_last=False,pin_memory=False,collate_fn=collate_fn_sparse_cite)
if train == 'multi':
    # train multi
    train_set = DatasetSparse_multi('../input/open-problems-msci-multiome-sparse-matrices/test_multi_inputs_sparse.npz','../input/open-problems-msci-multiome-sparse-matrices/train_multi_targets_sparse.npz')
    train_set_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, 
                                                   num_workers=2,drop_last=False,pin_memory=False,collate_fn=collate_fn_sparse_multi)

Finish loading data into memory


In [7]:
model_config = {
'input_dim':train_set.x.shape[1]+1,
'device':torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ,
'output':23418, # cite:140 multi:23418
'layer_one':256, # cite:160  multi:256
'fc_dims':[128,64,32],
'epochs' : 10
}
reg_model = LinearReg(model_config)
optimizer = torch.optim.SGD(reg_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0) # 5e-4
loss_fn = NegativeCorrLoss()

In [8]:
print(model_config)

{'input_dim': 228943, 'device': device(type='cuda', index=0), 'output': 23418, 'layer_one': 256, 'fc_dims': [128, 64, 32], 'epochs': 10}


In [9]:
for epoch in trange(model_config['epochs'],desc = 'epochs'):
    losses = []
    train_batch_iterator = tqdm(train_set_loader, disable=False, file=sys.stdout)
    for index,feats,values,labels in train_batch_iterator:
        index = index.to(model_config['device'])
        feats = feats.to(model_config['device'])
        values = values.to(model_config['device'])
        labels = labels.to(model_config['device'])

        optimizer.zero_grad() 
        # batch_size 最后一个不一样
        batch_size = len(labels)
        outputs = reg_model(batch_size, index, feats, values)

        loss  = loss_fn(outputs,labels)

        loss.backward() 
        optimizer.step() 

        losses.append(loss.item())
        train_batch_iterator.set_postfix(loss=loss.item()) # 实时 batch 的loss

    print('epoch:{} \n loss:{:.6}'.format(epoch,np.mean(losses)))

epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/874 [00:00<?, ?it/s]

epoch:0 
 loss:-0.629437


  0%|          | 0/874 [00:00<?, ?it/s]

epoch:1 
 loss:-0.658821


...

epoch:7 
 loss:-0.659285


  0%|          | 0/874 [00:00<?, ?it/s]

epoch:8 
 loss:-0.659302


  0%|          | 0/874 [00:00<?, ?it/s]

epoch:9 
 loss:-0.659317


In [10]:
torch.save(reg_model.state_dict(),'./reg_model_{}'.format(train))

In [11]:
del train_batch_iterator
gc.collect()
del train_set_loader
gc.collect()
del train_set
gc.collect()                                       

0

In [12]:
# train cite
if train == 'cite':
    test_set = DatasetSparse_pred('../input/torch-demo/test_cite_inputs_sparse.npz')
    test_set_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, 
                                                   num_workers=2,drop_last=False,pin_memory=False,collate_fn=collate_fn_sparse_pred)
    
if train == 'multi':
    test_set = DatasetSparse_pred('../input/open-problems-msci-multiome-sparse-matrices/test_multi_inputs_sparse.npz')
    test_set_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, 
                                                   num_workers=2,drop_last=False,pin_memory=False,collate_fn=collate_fn_sparse_pred)
    multi_test_row = np.load('../input/data-process/multi_test_row.npy')
    multi_test_row_name = np.load('../input/data-process/multi_test_row_name.npy')
    multi_test_col = np.load('../input/data-process/multi_test_col.npy')
    multi_test_col_name =pd.read_hdf('../input/open-problems-multimodal/train_multi_targets.h5',start=0,stop=1).columns.to_numpy()
    
    


with torch.no_grad():
    test_batch_iterator = tqdm(test_set_loader, disable=False, file=sys.stdout)
    test_pred = []
    start_r = 0
    start_c = 0
    for index,feats,values in test_batch_iterator:
        index = index.to(model_config['device'])
        feats = feats.to(model_config['device'])
        values = values.to(model_config['device'])

        # batch_size 最后一个不一样
        batch_size = index[-1].tolist()+1
        outputs = reg_model(batch_size, index, feats, values)
        
        outputs = outputs[multi_test_row[start_r:start_r+batch_size]]
        if len(outputs)>0:
            c_cnt = np.sum(multi_test_row[start_r:start_r+batch_size])
            index = torch.tensor(multi_test_col[start_c:start_c+c_cnt])
            index = index.to(model_config['device'])
            outputs = outputs.gather(1, index)
            start_r+=batch_size
            start_c+=c_cnt
            test_pred+=outputs.tolist()
        
test_pred = np.array(test_pred)
if train == 'cite':
    np.save('./cite_test_pred',test_pred)
if train == 'multi':
    np.save('./multi_test_pred',test_pred)

Finish loading data into memory


  0%|          | 0/874 [00:00<?, ?it/s]

In [13]:
del test_batch_iterator
gc.collect()
del test_set_loader
gc.collect()
del test_set
gc.collect() 
del index
gc.collect()
del outputs,feats,values
gc.collect()

0

In [14]:
# submit
eval_ids = pd.read_csv('../input/open-problems-multimodal/evaluation_ids.csv', index_col='row_id')

# Convert the string columns to more efficient categorical types
eval_ids = pd.read_csv('../input/open-problems-multimodal/evaluation_ids.csv', index_col='row_id')
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)

In [15]:
multi_test_row = np.load('../input/data-process/multi_test_row.npy')
multi_test_row_name = np.load('../input/data-process/multi_test_row_name.npy')
multi_test_col = np.load('../input/data-process/multi_test_col.npy')
multi_test_col_name =pd.read_hdf('../input/open-problems-multimodal/train_multi_targets.h5',start=0,stop=1).columns.to_numpy()

In [16]:
for idx, index in enumerate(tqdm(multi_test_row_name)):
    old = multi_test_col_name[multi_test_col[idx]]
    new = eval_ids.gene_id[eval_ids.cell_id == index].to_numpy()
    temp={name:i for i,name in enumerate(old)} 
    col = [temp[name] for name in new]
    row = test_pred[idx][col]
    submission.loc[index] = row

  0%|          | 0/16780 [00:00<?, ?it/s]

In [17]:
print('na:', submission.isna().sum())

na: 6812820


In [18]:
import pickle
submission.reset_index(drop=True, inplace=True)
submission.index.name = 'row_id'
with open("partial_submission_multi.pickle", 'wb') as f: pickle.dump(submission, f)